# About: 古い環境の削除

---

移行して利用しなくなった古い運用環境を削除します。

## 概要

古い運用環境のリソース（論理ボリューム、設定ファイル、コンテナイメージ）を削除します。

![古い運用環境の削除](images/moodle-193-01.png)

### グループ名の指定

このNotebookの操作対象となる UnitGroup名を指定します。

In [ ]:
# (例)
# ugroup_name = 'Moodle'

ugroup_name =

#### チェック

指定された `UnitGroup`名に対応する group_varsファイルが存在していることを確認します。エラーになる場合は、指定したUnitGroup名が正しくないか、「010-パラメータ設定.ipynb」のNotebookを実行していないことが考えられます。

In [ ]:
from pathlib import Path
if not (Path('group_vars') / (ugroup_name + '.yml')).exists():
    raise RuntimeError(f"ERROR: not exists {ugroup_name + '.yml'}")

UnitGroupに属する VCノードに対して Ansible で操作できることを確認します。

In [ ]:
!ansible {ugroup_name} -m ping
!ansible {ugroup_name} -b -a 'whoami'

検証環境に関するパラメータが group_vars に保存されていることを確認します。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

if 'previous_info_list' not in gvars:
    raise RuntimeError("ERROR")

### VCCアクセストークンの入力

VCCにアクセスするためのトークンを入力します。

In [ ]:
from getpass import getpass
vcc_access_token = getpass()

#### チェック

入力されたアクセストークンが正しいことを、VCCにアクセスして確認します。

In [ ]:
from vcpsdk.vcpsdk import VcpSDK
vcp = VcpSDK(vcc_access_token)

### 準備

これまでに他のNotebookで設定したパラメータを読み込む処理などを行います。

group_varsファイルに保存されているパラメータを読み込みます。

In [ ]:
%run scripts/group.py
gvars = load_group_vars(ugroup_name)

VCCのVault サーバにアクセスする際に必要となるパラメータを環境変数に設定します。

In [ ]:
import os
os.environ['VAULT_ADDR'] = vcp.vcc_info()['vault_url']
os.environ['VAULT_TOKEN'] = vcc_access_token

古い運用環境のコンテナが停止していることを確認するために、コンテナ状態を更新します。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    docker-compose up -d --remove-orphans'

削除対象となる環境が存在していることを確認します。

In [ ]:
candidate = [
    x['project_tag']
    for x in gvars.get('previous_info_list', [])
    if ('project_tag' in x and
        x['project_tag'] not in gvars.get('delete_tags', []))
]
if len(candidate) == 0:
    raise RuntimeError("ERROR")

## パラメータの設定

削除候補となる運用環境のタグの一覧を表示します。

In [ ]:
candidate = [
    x['project_tag']
    for x in gvars.get('previous_info_list', [])
    if ('project_tag' in x and
        x['project_tag'] not in gvars.get('delete_tags', []))
]

for tag in candidate:
    print(tag)

次のセルで削除対象となるタグの値をリストで指定してください。

In [ ]:
# (例)
# target_tags = [
#     '0',
#]

target_tags = [

]

指定したタグ名が妥当かチェックします。次のセルを実行してエラーになる場合は指定したタグの値が正しくありません。値を指定し直してください。

In [ ]:
if not all([x in candidate for x in target_tags]):
    raise RuntimeError("ERROR")

## 古い環境の削除

### 論理ボリュームの削除

現在の論理ボリュームを一覧表示します。

In [ ]:
!ansible {ugroup_name} -b -a 'lvs -S "lv_attr=~V" -O lv_name'

削除対象となる論理ボリュームを表示します。

In [ ]:
lvol_map = {'moodle': 'moodle', 'php': 'moodle', 'db': 'db'}
remove_vols = dict([
    (f'{tag}_{v}', vg)
    for v, vg in lvol_map.items()
    for tag in target_tags
])

for lv, vg in remove_vols.items():
    print(f'lv={lv} vg={vg}')

論理ボリュームを削除します。

In [ ]:
for lv, vg in remove_vols.items():
    print(f'lv={lv} vg={vg}')
    name = f'{vg}-{lv}{"-enc" if gvars[vg + "_volume_encrypt"] else ""}'
    !ansible {ugroup_name} -b -a 'umount /dev/mapper/{name}'
    if gvars[vg + "_volume_encrypt"]:
        !ansible {ugroup_name} -b -a 'cryptsetup close {name}'
    !ansible {ugroup_name} -b -m lvol -a 'vg={vg} lv={lv} state=absent force=yes'

削除後の論理ボリュームの一覧を表示します。

In [ ]:
!ansible {ugroup_name} -b -a 'lvs -S "lv_attr=~V" -O lv_name'

### 設定ファイルの削除

古い運用環境の設定ファイルの状況を確認します。

In [ ]:
target_dirs = ' '.join([
    f'{x}-{tag}'
    for x in ['moodle', 'db']
    for tag in target_tags])
print(target_dirs)
!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    tree {target_dirs}'

古い運用環境の設定ファイルを削除します。

In [ ]:
!ansible {ugroup_name} -b -a 'chdir=/opt/moodle \
    rm -rf {target_dirs}'

削除されたことを確認します。

> 削除されていれば`[error opening dir]`と表示されます。

In [ ]:
!ansible {ugroup_name} -a 'chdir=/opt/moodle \
    tree {target_dirs}'

### コンテナイメージの削除

古い運用環境で利用していたコンテナイメージの一覧を表示します。

In [ ]:
target_images = [
    x[key]
    for x in gvars['previous_info_list']
    if ('project_tag' in x and
        x['project_tag'] in target_tags)
    for key in x.keys()
    if key.endswith('_image_name')
]
for img in target_images:
    !ansible {ugroup_name} -a 'docker images {img}'

古い運用環境のコンテナイメージを削除します。

In [ ]:
!ansible {ugroup_name} -a 'docker rmi {" ".join(target_images)}'

コンテナイメージが削除されたことを確認します。

> 削除が成功していれば、ヘッダーのみが表示されます。

In [ ]:
for img in target_images:
    !ansible {ugroup_name} -a 'docker images {img}'

### group_varsの更新

古い運用環境を削除したことを `group_vars`ファイルに記録します。

In [ ]:
x = gvars.get('delete_tags', [])
x.extend(target_tags)
update_group_vars(ugroup_name, delete_tags=x)